# KEY POINTS FROM THE COURSE LESSONS

## chin_3_Linear_Reggression_Consulting_Project.ipynb
------------------------------------------------------------------------------
PURPOSE
* This project is to determine number of crew members needed for future ships being built.
* Given are various parameters of the ship like number of cruise_line, passengers, cabin count, age of ship, length and weight of ship etc.
* Also given the condition that the number of crews may depend on the cruise_line name.

STEPS:
* We are going to create a model to predict 'crew' which is the crew count.
* This has a dependency on a string field 'cruise_line'.
* For the Spark ML algorithm to work we need to prepare the data consisting of two columns:
>1. The features column should be a vector containing details from all the desired numeric fields of the input data and 
>2. a label field, the 'crew' numeric field to predict.
* 'Cruise_line' string field being one of the predictors, we need to convert it from categorical value to a numeric field using "StringIndexer".
>* str_Indexer = StringIndexer(inputCol='Cruise_line', outputCol='Cruise_line_indexed')
>* sdf_cruises_indexed = str_Indexer.fit(sdf_cruises).transform(sdf_cruises)
* Then use this indexed field along with other participating numeric fields to generate the vectorized 'features' column, using VectorAssembler, for spark ML libs to process further.
>* assembler = VectorAssembler(inputCols=feature_input_col_list, outputCol='features')
>* sdf_cruises_indexed_vec_data = assembler.transform(sdf_cruises_indexed)
* Pull out the vector column ('features') amd label columne ('crew') into a dataframe and that will be inut to the spark ML library.
* split this final data into training data and test data.
>* final_data = sdf_cruises_indexed_vec_data.select('features', 'crew')
>* train_data, test_data = final_data.randomSplit([0.7, 0.3])
* Train the LinearRegression (lr) instance with training data i.e. lr_model = lr.fit(training_data)and 
>* lr_ship = LinearRegression(featuresCol='features', labelCol='crew', predictionCol='prediction')
>* lr_ship_model = lr_ship.fit(train_data)
* Evaluate it on test data i.e. test_Result = lr_model.evaluate(test_data)
* Cross validate the error metrics from the result (test_result.rootMeanSquaredError, .r2, .residuals.show() ) and from model print .coefficients, .intercept etc
* The R-squared (i.e. .r2) was a bit higher side indicating a good fit, hence we became suspicious and tried to find the cause of good result by trying to find the correlation of the label column ('crew') with few of the ship features like 'passengers' and 'cabins' in the original input dataframe. We found the corr values to be high in the original data and hence the R-squared indicated a good fit.

## chin_2_Linear_Regression_Code_Along.ipynb
-----------------------------------------
* Decide which numeric columns will be treated as features and which numeric field will be our label, based on the oinput data.
* Create a VectorAssembler from the feature columns ([inputCols]) to get a combined vectorized column (outputCol)
* Then invoke transform method using vectorAsembler on the input dataframe which will add a new vectorized column to the dataframe and that new field will have the data from all the "inputCols"
* Now extract a dataframe with the vector column and label columns as this two columns are only needed by spark MLlib to predictionusing Supervised model. This is our final_data for input to our model.
>* Create a Vector assembler with a list parameter containing the names of numeric columns and a outputCol holding th ename of the vectorized column that will be added to the result by this assempber.transform() method.
* Get the combined merged feature column and the label field as a two-column dataframe and call it 'final_data' to be passed into our model for further operations.
* Split the final input data into train_data and test_data in 70:30 ratio.
* Train the model on train_data, evaluate the model on test_Data. Let the test_results.residuals be the evaluated labels.
* Check the accuracy performance of the model evaluation by checking various parameters of test_result, such as .r2.residuals, rootMeanSquaredError, .meanSuaredError, .meanAbsoluteError etc.
* We don't ave any unlabeled data, so we will make the test_data unlabeled so as to predict the labels for those features. As this test_data is not used in training the model, model is not aware of this.
* Predict the labels for this unlabeled dataand compare the 'prediction' column of the predicted_result against the test_result,r2,residuals.
* Here I have used sparkSession.sql ("SELECT.  .. .. ") beautifully to combine data from multiple dataframes and to find the deviations in our predictions outout from the actual data in test_data. For this we first registered the participating dataframes as views and then performed complex select statements using these views.

## chin_1_Linear_Regression_Example.ipynb
-----------------------------------------
* Loading libsvm formatted text files (contians the verctorized text data using sparkSessn.read.format('libsvm').load("libsvm_formatted_text_file.txt")
* The libsvm formatted text file does nto have any column name defined in the file. It contained 'label' in the first column and the trailing columns were in the format 1:float1 2:float2 3:float3 etc. the libsvm format reader of sparkSessin automatically merges all these training columns intelligently and creates a merged column that contains te veector of all these feature columns
* In real scenarios we need to prepare the vector column for all features merged into one using VectorAssembler (explained in next chapter)
* For me direct session.read.format().load() did not work and I had to use session.read.format('libsvm').option('numFeatures', nFeatrureCount).load(...)
* Split this data in 70:30 ratio to traindata and test_data respectively
* Instantiate a LinearRegression model and train or fit it with training data.
* Now evaluate the model against test data, which compares the predicted labels (from te result test_result) against the labels in test_data.
* Check the results by using test)result, .rootMeanSquaredError, .r2.residuals and fw other error metrics such as . meanSquaredError, .meanAbsoluteError etc.
* Create a dataset without labels (prepare mimiced data from test_data, which is not seen by the model), by stripping off the labels from test_Data and applying model.transform on this unlabeled data to get the prediction values.

## chin_2_Spark DataFrame Basics-withGroupBy.ipynb
---------------------------------------------------------------------------
* printing formatted text in python using '\\x1B\[Nm' where N is a integer (check chinmay_tools.py)

* Converting Pandas DataFrame to Spark DataFrame and vice versa. (check chinmay_tools.py)
* Using inferSchema attribute during spark DataFrame creation from file to decide teh data types automatically.
* Additionally we can customize json schema to change the datatypes in dataframe while loading from json files (repeat)
* Renaming a column in a DataFrame using sdf.withColumnRename(old_col_name, new_col_name)
* Adding or replacing a column with a different computed value or a masked value e.g. using sdf.withColumn(colName, sdf['col3']+5) or replacing with a string literal using .lit(const_string) function
* Using groupBy() on spark DataFrames.
* Using aggregates on groupBy objects.
* If a DataFrame has multipel numeric fields then we can apply multiple aggregates on a single groBy object with one aggr fn per numeric column.
* If multiple aggr fns are applied on a single column then last function on that column is resulted and all previous funcs on that column are ignored. grpByCompany.agg({'col1':'aggfn1', 'col2':'aggfn2', 'col3':'aggfn3'})
* Sorting dataframe in ascending order by using orderBy with column name. sdf.orderBy('col1') sorts in ascending order of col1.
* Sorting dataframe in descending order by using orderBy with column (but not column name). sdf.orderBy(sdf['col1'].desc()) sorts in descending order of col1.
* If we want to sort a dataframe with different orders for each field, then we must use columns for descending ordering but we can use either columns or just column names for thecolumns with ascending order.`
* Handling missing data in a DataFrame
* Dropping the records if all fields are null, or if any field is null or there are less than n null fields (thres=n). sdf.na.drop(...), "subset" attribute can decide which columns to check for null. Here null means na.
* Replacing numm values with specified string using sdf.na.fill(...), for any field, for the field / fields listed suing subset or list array. Also replacing with a computed value from an aggregate result of remaining nun-null values.
* Handling Date fields using functions from pyspark.sql.functions e.g. year(), month() etc.. or date_format('date_field_name', my_format_string)

## chin_1_Spark DataFrame Basics.ipynb
--------------------------------------------------------
* Using modules or functions from another custom python file by adding th efile path to "sys.path"
* Using modules or functions from another ipynb jupyter notebook (less convenient)
* Getting help on any mdule that we are using through question marks
* Using a custom defined schema (to change data type) while importing a json into a spark DataFrame
* Using dataframe.select().where(), dataframe.where(), dataframe.filter() to get subsets of a dataframe with filtered conditions
* Using sparkSessn.sql() to get results from multiple DataFrames by registering the dataframes as views and performing compex select statements on those views.


## chinmay_tools.ipynb
-------------
* Generating python file from jupyter notebook files (.ipynb) using "ipynb-py-convert" application (installe using pip)
* Generating html or pdf files from jupyter notebook files using syntax "jupyter nbconvert -to=html test.ipynb", which produces test.html. To generate pdf, print the html using system wide pdf rinter. Don't use "--to=pdf" which needs a lot of tools to be installed.
* Using our python modules or functions in another python / ipynb notebook by adding the fodler location of the python file to "sys.path" list
* Enabling notebook shells to display multiple print results (without using explicit print statement by setting InteractiveShell.ast_node.interactivity to "all".
* Print formatted text on console concatenating "\\x1B\[Nm" and "\\x1B\[0m" at begining and at end of a string.
* Printing contents of a text file (.txt, .csv, .xml etc)
* Convert Pandas DataFrame into a Spark DataFrame and vice versa.
* Convert the contents of a Spark DataFrame into Json.
* Mask certain colums of a Spark DataFrame using lit() function and sdf.withColumn() function.
* Converts the content of a xml into a Pandas DataFrame. The xml format is hardcoded to the call log xml generated by an android applicaiton named "SuperBackUp".
* Print some LaTeX characters
* Print multiple header lines like H1, H2, H3 etc in a single line using <i>'style="display: inline"'</i> css style